# Bird Species Classier 
Built using the [Kaggle dataset](https://www.kaggle.com/gpiosenka/100-bird-species) by [Gerald Piosenka](https://www.kaggle.com/gpiosenka)

Project By: [Uzair](https://github.com/Uzair05)

## Download, extract and arrange data.

In [ ]:
#!pip install --upgrade kaggle
!kaggle datasets download -d gpiosenka/100-bird-species

In [ ]:
import os 
import zipfile

try:
    os.mkdir("./data/")
except Exception as err:
    print(f"Error in creating data/:\t{err}")


localzip = "./100-bird-species.zip"
zip_ref = zipfile.ZipFile(localzip, 'r')
try:
    zip_ref.extractall('data/')
except Exception as err:
    print(f"Erorr in zip extraction:\t{err}")
finally:
    zip_ref.close()

## Handle Data
Create the train, test and valdation data.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

In [ ]:
train_dir = "./data/train"
test_dir = "./data/test"
valid_dir = "./data/valid"

train_datagen = ImageDataGenerator(
    rescale = 1./255., 
    rotation_range = 20, 
    shear_range = 0.2,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    zoom_range = 0.2, 
    horizontal_flip = True, 
    vertical_flip = True, 
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1./255.
)
valid_datagen = ImageDataGenerator(
    rescale=1./255.
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(128,128), 
    batch_size=20, 
    class_mode="categorical", 
)
test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(128,128), 
    class_mode="categorical", 
    batch_size=20
)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir, 
    target_size=(128,128), 
    class_mode="categorical", 
    batch_size=20
)

## CNN Transfer Learning - Using Architecture with pretrained weights.
Architecture was provided by the creator of the dataset.


In [ ]:
pretrained_model = tf.keras.models.load_model("./data/EfficientNetB3-birds-98.92.h5")
pretrained_model.summary()

In [ ]:
for layers in pretrained_model.layers:
    layers.trainable = False
pretrained_model.trainable = False

last_layer = pretrained_model.get_layer("top_activation")
last_output = last_layer.output

In [ ]:
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(256, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(315, activation=tf.nn.softmax)(x) # To classify 315 species.

model = tf.keras.Model(pretrained_model.input, x)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

## CNN Transfer Learning - Creating new Architecture using EfficientNet
I wil try to create and train my own architecture

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB3

In [ ]:
pretrained_model = EfficientNetB3(
    include_top=False, 
    weights="imagenet", 
    input_shape=(128, 128, 3)
)

for layers in pretrained_model.layers:
    layers.trainable = False
pretrained_model.trainable = False

pretrained_model.summary()

In [ ]:
last_layer = pretrained_model.get_layer("top_activation")
last_output = last_layer.output

In [ ]:
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(256, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(315, activation=tf.nn.softmax)(x) # To classify 315 species.

model = tf.keras.Model(pretrained_model.input, x)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

## Fit and Evaluate

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        accuracy = 0.95 # Percentage Accuracy.
        if(logs.get('accuracy') != None) and (logs.get('accuracy') >= accuracy): # Experiment with changing this value
          print(f"\nReached {accuracy*100}% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

In [ ]:
history = model.fit(
    train_generator,
    epochs = 5, 
    #steps_per_epoch = 50, 
    validation_data = test_generator, 
    verbose = 1, 
    validation_steps = 10,
    callbacks=[callbacks]
)

### Save Model

In [ ]:
tf.keras.models.save_model(model, filepath="my_model/EfficientNetB3_myown_untrained.h5", overwrite=True, save_format='h5')

### Print Model History

In [ ]:
history.history

### Evaluate Model Using Validation Dataset

In [ ]:
eval = model.evaluate(
    valid_generator,
    batch_size=20
)